# MEWpy Community Modeling

Author: Vitor Pereira, inspired on the work by Daniel Machado. 

License: [CC BY-SA 4.0](http://creativecommons.org/licenses/by-sa/4.0/)

-------

In this tutorial:

- You will learn how to perform flux balance analysis of microbial communities
using a model of the [central carbon metabolism of *E. coli*](https://journals.asm.org/doi/10.1128/ecosalplus.10.2.1).


## Install requirements 
To run this notebook we firstly need to install the required packages

In [ ]:
! pip install -U -q mewpy cplex escher

Verify the instalation

In [ ]:
import mewpy
mewpy.info()

IMPORTANT: The notebooks require a MEWpy version >= 0.1.26

### Run in Google colab

If you are running this notebook in Colab, you need to perform the following steps, otherwise skip.

In [ ]:
%%bash
[[ ! -e /colabtools ]] && exit
! pip install -U -q PyDrive

In [ ]:
if 'google.colab' in str(get_ipython()):
    from pydrive.auth import GoogleAuth
    from pydrive.drive import GoogleDrive
    from google.colab import auth
    from oauth2client.client import GoogleCredentials

    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)

    model_file = drive.CreateFile({'id':'1o0XthuEOs28UJ4XTa9SfFSFofazV-2nN'})
    model_file.GetContentFile('e_coli_core.xml.gz')

## Setting up a community

We will create a synthetic microbial consortium with two *E. coli* mutants growing in minimal medium. In one of the mutants we will knockout the glucose transporter and in the other we will knockout the ammonium transporter.

In [ ]:
from cobra.io import read_sbml_model
from mewpy import get_simulator

model = read_sbml_model('e_coli_core.xml.gz')
wildtype = get_simulator(model)
solution = wildtype.simulate()
print(solution)
solution.find('EX')

Now we create our two mutants (`glc_ko` and `nh4_ko`):

In [ ]:
glc_ko = wildtype.copy()
glc_ko.id = 'glc_ko'
glc_ko.set_reaction_bounds('GLCpts', 0, 0)

In [ ]:
nh4_ko = wildtype.copy()
nh4_ko.id = 'nh4_ko'
nh4_ko.set_reaction_bounds('NH4t', 0, 0)

## Comparing models

Community models require that metabolites have the same identifiers accros all models. MEWpy offers some functions tho that end, computing the metabolites, reactions and uptakes overlaps between a list models.

In [ ]:
from mewpy.cobra.com import *
mets, rxns, over = jaccard_similarity_matrices([glc_ko, nh4_ko])

In [ ]:
mets

In [ ]:
rxns

In [ ]:
over

## Building communities

**MEWpy** has some basic functionality for working with microbial communities, one is the `CommunityModel` class to create microbial communities from a list of models of individual species: 

In [ ]:
from mewpy.model import CommunityModel
community = CommunityModel([glc_ko, nh4_ko],flavor='cobra')

In [ ]:
sim = community.get_community_model()

This community model ignores the environmental conditions that were specified in the original models (since these could be very different). 

To make our life easier, we will extract the nutrient composition specified in the wild-type model to use later.

In [ ]:
from mewpy.simulation import Environment
M9 = Environment.from_model(wildtype)
M9

## Simulation using FBA

A very simple way to simulate a microbial community is to merge the individual models into a single model that mimics a "super organism", where each microbe lives inside its own compartment, and run a (conventional) FBA simulation for this *super organism*.

In [ ]:
solution = sim.simulate(constraints=M9)

print(solution)
solution.find('EX')

We can see that the model predicts a growth rate (total biomass per hour) similar to the wild-type, with an efficient consumption of glucose and ammonia that results in respiratory metabolism.

But what is each organism doing, and are both organisms actually growing at the same rate?

Let's print the biomass flux for each organism:

In [ ]:
solution.find('BIOMASS', sort=True,show_nulls=True)

and all non null fluxes by organism:

In [ ]:
solution.find('glc_ko', sort=True)

In [ ]:
solution.find('nh4_ko', sort=True)

Actually it seems that only one of the organisms is growing while the other has an active metabolism (it exchanges metabolites with the environment and with the other organism) performing the role of a bioconverter, but none of the flux is used for growth. 

> Do you think this would be a stable consortium ?

## Community Simulation with SteadyCom

**SteadyCom** by [Chan, et al (2017)](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1005539) is a recent community simulation method that takes into account the fact that to reach a stable composition the organisms need to grow at the same *specific growth rate* (1/h), which means that the *absolute growth rate* (gDW/h) of each organism is proportional to its *abundance* at steady-state (gDW).

Let's simulate the same community using SteadyCom:

In [ ]:
from mewpy.cobra.com import SteadyCom as SteadyCom1
solution = SteadyCom(community, constraints=M9)


In this case the solution object shows the overall community growth rate and the relative abundance of each species:

In [ ]:
solution

The `solution` object for community simulations implements a few additional features, such as enumerating all the cross-feeding interactions:

In [ ]:
solution.cross_feeding(as_df=True).dropna().sort_values('rate', ascending=False)

We can plot the fluxes of each mutant in a map to help with interpretation of the results:

In [ ]:
from mewpy.visualization.escher import build_escher
if 'google.colab' in str(get_ipython()):
    from google.colab import output
    output.enable_custom_widget_manager()

build_escher(fluxes=solution.internal['glc_ko'])

In [ ]:
build_escher(fluxes=solution.internal['nh4_ko'])

## Explore alternative solutions

Unfortunately, one limitation of **SteadyCom**, which is exemplified by [Chan, et al (2017)](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1005539) in Figure 3 (reproduced below), is the variability in the solution space when the community is not growing at the maximum (theoretical) growth rate.

> Would you expect a synthetic community to grow at its maximum growth rate?

**MEWpy** implements a variability analysis function for the SteadyCom solution space, let's see what happens if the community is growing at 90% of the theoretical maximum:

In [ ]:
from mewpy.cobra.com import SteadyComVA
variability = SteadyComVA(community, obj_frac=0.9, constraints=M9)

print('Strain\tMin\tMax')
for strain, (lower, upper) in variability.items():
    print(f'{strain}\t{lower:.1%}\t{upper:.1%}')

As you can see, there is a really large variability in this solution space. This means that we know in theory the two mutants **can** cooperate and survive in minimal media, but there is still a lot of uncertainty with regard to **how** they will achieve a stable consortium.

> How do you think we can reduce this uncertainty?

Firstly, lets set the environment conditions:

In [ ]:
sim.set_environmental_conditions(M9)

We may now impose constraints on each organism growth, such as stating that each organism need to grow at least 0.1/h

In [ ]:
constraints={community.organisms_biomass['nh4_ko']:(0.1,1000), 
             community.organisms_biomass['glc_ko']:(0.1,1000)}
solution = sim.simulate(constraints=constraints)
solution

In [ ]:
solution.find('BIOMASS')

Alternatively, we might choose to impose relative growth rates for each of the organisms:

In [ ]:
community = CommunityModel([glc_ko, nh4_ko],
                           add_compartments=True,
                           merge_biomasses=True,
                           flavor='cobra')

In [ ]:
sim = community.get_community_model()
sim.set_environmental_conditions(M9)

In [ ]:
solution = sim.simulate()
print(solution)
solution.find('BIOM')

In [ ]:
sim.find(community.biomass)

The relative abundance (relative growth rates) are by default equal. We may though change these ratios:  

In [ ]:
community.set_abundance({'glc_ko':1,'nh4_ko':2.5})
sim.simulate().find('BIOMASS')

## SMETANA

**SMETANA** implements several algorithms to analyse cross-feeding interactions in microbial communities. These have been describe in [Zelezniak et al, PNAS (2015)](https://www.pnas.org/doi/abs/10.1073/pnas.1421834112). Please read the paper for a more detailed explanation.

SCS (species coupling score): measures the dependency of one species in the presence of the others to survive

In [ ]:
sc_score(community)

MUS (metabolite uptake score): measures how frequently a species needs to uptake a metabolite to survive

In [ ]:
MUS = mu_score(community)
MUS

In [ ]:
MUS.glc_ko

In [ ]:
MUS.nh4_ko

MPS (metabolite production score): measures the ability of a species to produce a metabolite

In [ ]:
MPS = mp_score(community,environment=M9)
MPS

MRO (metabolic resource overlap): calculates how much the species compete for the same metabolites.

In [ ]:
mro_score(community,environment=M9)